# Intro to Linear Regression Using Gradient Descent - Practical

* Linear regression is one of the most widely used techniques in statistical modeling. 

* It is a method used to model the relationship between a dependent variable and one or more explanatory variables. 

* The goal of linear regression is to find the line or hyperplane of best fit that accurately represents the relationship between the variables. 

In the lecture notes, we computed the weights for the best fit for the univariate case, using gradient descent. We also introduced the slight variations in the algebra for computing the weights for best fit for the multivariate case. 

In this practical, we will implement linear regression for the multivariate case, using synthetic data.

## Generating our data

Here we provide the code to generate data with 2 independently normally explanatory variables, with additive random noise. 

In [ ]:
import numpy as np

# Generate data with 2 features and a target variable
def generate_data_3d(n):
    np.random.seed(n)
    X = np.stack([np.random.normal(scale=2, size=n), np.random.normal(scale=5, size=n)]).T
    beta = np.absolute(np.random.randn(3))
    # add slight some standard normal noise
    y = beta[0] + beta[1]*X[:,0] + beta[2]*X[:,1] + np.random.normal(scale=1, size=n)
    return X, y, beta

n = 100
X, y, beta_true = generate_data_3d(n)

X.shape

As we control the data generation process, we know the true parameters for the linear model:

In [ ]:
print(f'True parameter values: b: {beta_true[0]:.2f}, weight 1: {beta_true[1]:.2f}, weight 2: {beta_true[2]:.2f}')

### Visualisation of 3D data

We can use the [*plotly*](https://plotly.com/python/) library to plot an interactive 3D scatter of our data. Plotly works really well at generating interactive graphs and visualisations. 

The [*express*](https://plotly.com/python/plotly-express/) module is designed to be very high-level and easy to use.

In [ ]:
import plotly.express as px
fig = px.scatter_3d(x=X[:, 0], y=X[:, 1], z=y, labels = { 'x':'x1', 'y':'x2', 'z':'y'})
fig.show()

## Multivariate linear regression - gradient descent

Now that we have more than one feature, our linear regression model will have more than one weight. 

We can therefore generate our predictions as:
$$
\hat{y_i} = b + w_1 x_{1,i} + w_2 x_{2,i}
$$

We can first try to find the parameters separately by thinking about multivariate linear regressions as a series of independent 1D linear regressions, and using gradient descent to find the parameters. Try adapting the univariate gradient descent code from the lecture notes to work for two $x$ features!

*Hint: As we assume the impacts of the features in $x$ on the target $y$ are independent, we can calculate the gradients at each iteration ignoring all other parameters*

In [ ]:
# Extract each feature as separate vectors 
x1 = X[:,0]
x2 = X[:,1]

# Initial values for the weight and bias
b = 0
w1 = 0
w2 = 0

eta = 0.001  # The learning rate
iterations = 1000  # The number of iterations to perform gradient descent

n = float(len(x1)) # Number of observations in X

# Performing Gradient Descent 
for i in range(iterations): 
    # write your code here
    y_pred = b + w1*x1 + w2*x2 # The current predicted value of Y
    D_w1 = (-2/n) * sum(x1 * (y - y_pred))  # Derivative wrt w_1
    D_w2 = (-2/n) * sum(x2 * (y - y_pred))  # Derivative wrt w_2
    D_b = (-2/n) * sum(y - y_pred)  # Derivative wrt b
    w1 = w1 - eta * D_w1  # Update w_1
    w2 = w2 - eta * D_w2  # Update w_2
    b = b - eta * D_b  # Update b
    
print(f'Estimated parameters: bias: {b:.2f}, weight 1: {w1:.2f}, weight 2: {w2:.2f}')

These are not too far away from our true values - not bad! 

But how can we know when we have fully converged?

Try playing around with values for the learning rate (eta) and number of iterations. 

* What happens if you increase the learning rate? (e.g. eta = 0.1)

    * Why?

* What happens if you decrease the number of iterations? (e.g. iterations = 100)

    * Why?

## Linear algebra to the (partial) rescue!

Whilst our code above was able to estimate usable parameters, it is quite clunky. 

We can make the code much simpler by treating $X$ as a *matrix* and using *linear algebra!*

### Accounting for the bias term


In order to perform the matrix multiplication and compute the gradient effectively in the multivariate case, we augment $X$ with a column of ones to account for the bias term. 

Call the new matrix `Xmat`

*Hint: You can use `np.hstack` to stack a vector of ones on an existing array.*

In [ ]:
# Add a dim to x with ones to account for the bias term
Xmat = np.hstack((np.ones((100,1)), X))


Now when computing `y_pred`, the updates for `beta`, and the error term, we do not need to account for the bias separately.

### Parameters as a vector

Instead of calculating each parameter separately, we can instead now calculate our *vector* of parameters $\beta$ at the same time, including the bias term. As such, we need to initialise our parameters to a vector of zeros.

In [ ]:
# Initial values for the weights (which include the bias term or beta_0)
# beta = ...
beta = np.zeros(Xmat.shape[1])


### Gradient descent with linear algebra 

Now try rewriting the gradient descent code to work with linear algebra. 

Recall that we can rewrite our prediction formula as: 

$$
\hat{y}=\beta^T X
$$

This can be achieved with the notation `y_pred = X@w`.

*Hint: Make sure you are using* `Xmat` *and not* `X`*!*

*Additional hint: to multiply our residuals by* `X` *in linear algebra, you may first need to take the transpose of* `X`

In [ ]:
# Linear algebra gradient descent
beta = np.zeros(Xmat.shape[1])
eta = 0.001  # The learning rate
iterations = 1000  # The number of iterations to perform gradient descent

n = float(Xmat.shape[0]) # Number of observations in X

# Performing Gradient Descent 
for i in range(iterations): 
    # write your code here
    y_pred = Xmat@beta  # The current predicted value of Y
    D_beta = -(2/n) * Xmat.T @ (y - y_pred)  # Derivative wrt beta
    beta = beta - eta * D_beta  # Update beta
print(f'beta: {beta}')

Note these are the exact same parameters we had before, but the code to get there is much neater!

In [ ]:
np.isclose(beta, [b, w1, w2])

### Visualising model hyperplane

How does our model actually look?

Given we have two gradients ($\beta_1$ and $\beta_2$) and an intercept ($\beta_0$), we can see the model defines a 2D plane in 3D. 

In [ ]:
import plotly.graph_objects as go

fig1 = px.scatter_3d(x=Xmat[:, 1], y=Xmat[:, 2], z=y, labels = { 'x':'x1', 'y':'x2', 'z':'y'})

# Create a mesh grid to get predictions for and plot a hyperplane
xy = np.array([[i,j] for i in np.linspace(-4,4,20) for j in np.linspace(-15,10,60)])
y_plane = np.hstack((np.ones((1200,1)), xy))@beta

fig2 = go.Scatter3d(x=xy[:,0], y=xy[:,1], z=y_plane, opacity=0.7,  mode='markers',
    marker=dict(
        size=4,
        color=np.ones_like(y_plane),
        colorscale='Viridis',
        opacity=0.8
    )
)
fig3 = go.Figure(data=(fig2,)+ fig1.data)

fig3.update_layout(
    scene=dict(
        xaxis_title='x1',
        yaxis_title='x2',
        zaxis_title='y',
    )
)

fig3.show()

More generally, for $J$ input features (i.e. a $J$-dimensional feature space), the linear regression model represents a $J$-dimensional hyperplane in $J+1$-dimensional space.

## Summary

In this practical you have implemented linear regression using gradient descent, and then improved the code using linear algebra. 

However, we can actually use linear algebra to find our linear regression parameters directly (i.e. without needing to use gradient descent), using a technique known as *Ordinary Least Squares (OLS)*.